In [2]:
import sys
sys.path = ['', '/Users/jiazhuoqin/anaconda/lib/python27.zip', '/Users/jiazhuoqin/anaconda/lib/python2.7', '/Users/jiazhuoqin/anaconda/lib/python2.7/plat-darwin', '/Users/jiazhuoqin/anaconda/lib/python2.7/plat-mac', '/Users/jiazhuoqin/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/jiazhuoqin/anaconda/lib/python2.7/lib-tk', '/Users/jiazhuoqin/anaconda/lib/python2.7/lib-old', '/Users/jiazhuoqin/anaconda/lib/python2.7/lib-dynload', '/Users/jiazhuoqin/anaconda/lib/python2.7/site-packages', '/Users/jiazhuoqin/anaconda/lib/python2.7/site-packages/Sphinx-1.6.3-py2.7.egg', '/Users/jiazhuoqin/anaconda/lib/python2.7/site-packages/aeosa']
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from sklearn import linear_model
#reload(sys) 
#sys.setdefaultencoding('utf8') 

# read data

In [3]:
import pandas as pd
df = pd.read_csv('submissions.csv', encoding='utf-8', names = ['image_id','unixtime','rawtime','title','total_votes','reddit_id','number_of_upvotes',\
'subreddit','number_of_downvotes','localtime','score','number_of_comments','username',\
'undefined1','undefined2', 'undefined3']

)

/Users/jiazhuoqin/anaconda/envs/gl-env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,1,4,6,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.iloc[1:, :]

In [10]:
titles = df['title'].tolist()
subreddit = df['subreddit'].tolist()
score = df['score'].tolist()

# get x and subreddit

In [6]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# list for tokenized documents in loop
texts = []

# loop through document list
for i in titles:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
  
    # stem tokens
    try:
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    except:
        print i
    # add tokens to list
    texts.append(stemmed_tokens)  #data 1 texts

In [32]:
from collections import defaultdict
count = defaultdict(int)

for document in texts:
    for stem in document:
        count[stem] += 1
        
for letter in list('abcdedfgijklmnopqrstuvwxyz'):
    if letter in count:
        count.pop(letter)
        
words_cut = [(item[1] , item[0]) for item in count.items() if item[1] > 10]
words_sort = sorted(words_cut,reverse = True)
len_words = len(words_sort) # data 3 len of frequent words
print len(words_sort)

words_cut_sorted = [item[1] for item in words_sort]  # data 2 frequent words(stems)

4577


# use frequent stem to predict score (influence of stem)

In [33]:
X_stem = []
for document in texts:
    temp = [1] + [0] * len_words
    for item in document:
        if item in words_cut_sorted:
            temp[words_cut_sorted.index(item) + 1] += 1
    X_stem.append(temp)

In [34]:
#score has been got
Y_score = []
for item in score:
    Y_score.append(int(item))
print len(score)
print score[1]

132307
23


In [ ]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X_stem, Y_score) # when using all set of data, MEMORY ERROR!!!
theta = clf.coef_

In [44]:
theta = theta.tolist()

In [45]:
#store word-theta pairs
f = open('theta.txt','w')
max1 = max(theta)
min1 = min(theta)
max_index = theta.index(max1)
min_index = theta.index(min1)

f.write('the max theta is ' + words_cut_sorted[max_index] + ': ' + str(theta[max_index]) + '\n')
f.write('the min theta is ' + words_cut_sorted[min_index] + ': ' + str(theta[min_index]) + '\n')
for x in range(len(theta)):
    try:
        f.write(words_cut_sorted[x] + ': ' + str(theta[x]) + '\n')
    except Exception as e:
        pass

f.close()

# check result's R^2 and mse

In [46]:
mse = 0

for i in range(132307):
    temp = 0
    for j in range(len_words + 1):
        temp += X_stem[i][j] * theta[j]
    mse += (Y_score[i] - temp) ** 2
mse = mse *1.0 / 132307
print mse
print clf.score(X_stem, Y_score, sample_weight=None)

217584.005219
0.0600356785449


# check influence

In [25]:
print len(theta)
print len(words_cut_sorted)
print theta[:100]
print words_cut_sorted.index(u'dog')

900
899
[ 243.79549109    9.33716888  -36.6641914   -27.60457848  -19.00734995
   -4.72793876  -19.45460056   -6.32457961  -26.9228977     7.90470621
    6.85685518  -15.94852879   41.29603368  -14.97874384   -4.06019382
  -17.92337457  -17.83340635   -0.67876939   26.55750396   47.89755628
  -31.45924949  -19.7875087    11.72490203   -6.11687528   34.80934735
  -22.04414761   -3.60983512    7.06458181   13.46278737  -17.61563304
  -11.22274791    1.12022952    4.01644004   20.06954561  -93.04873768
  -13.02918789  -21.32921893    9.13645911   23.09404815  -15.4077732
   58.89583205   17.90348274  -35.53761995   -1.91065149   24.52194482
   -9.27750016  -14.55793172    3.34561524    7.33391673  -29.6799541
   17.41247944   -0.97292458   -1.45907993  -48.89434959   22.44322598
  -33.07361742  -15.55227514  -26.96164552   29.35984779  -50.90504698
    9.24708458    7.89510548   24.8929253   -73.22655481   16.38576638
   15.83901878  -16.24412451  -89.77765077  -50.02427683  148.60101173
